In [1]:
!featurize dataset download 910888ea-f616-4cd9-bfd1-4acc754d6fae

Traceback (most recent call last):
  File "/environment/miniconda3/bin/featurize", line 8, in <module>
    sys.exit(cli())
  File "/environment/miniconda3/lib/python3.10/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/environment/miniconda3/lib/python3.10/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/environment/miniconda3/lib/python3.10/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/environment/miniconda3/lib/python3.10/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/environment/miniconda3/lib/python3.10/site-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/environment/miniconda3/lib/python3.10/site-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/environment

In [2]:
!pip install surprise

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
!pip install openpyxl

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [4]:
import pandas as pd
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
import numpy as np
import os

In [17]:
# Load the data
data_df = pd.read_excel('[final] April 2015 to Nov 30 2019 - Transformed Jester Data - .xlsx')
data_df.columns = range(data_df.shape[1]) # Rename columns to match with jokes dataframe

# Convert data_df from wide format to long format
data_df = data_df.stack().reset_index()
data_df.columns = ['user_id', 'joke_id', 'rating']

# Filter out missing ratings
data_df = data_df[data_df['rating'] != 99.0]

# Load the jokes
jokes_df = pd.read_excel('Dataset4JokeSet.xlsx')
jokes_df.columns = ['joke']
jokes_df.index.name = 'joke_id'

# Create a Reader object
reader = Reader(rating_scale=(-10, 10))

# Load the data into a Dataset object
data = Dataset.load_from_df(data_df[['user_id', 'joke_id', 'rating']], reader)

In [19]:
data_df

,user_id,joke_id,rating
0,0,0,1.00
72,0,72,8.21
159,1,0,4.00
178,1,19,-5.98
194,1,35,-4.47
...,...,...,...
1223821,7696,157,0.00
1223822,7696,158,0.00
1223823,7697,0,2.00
1223842,7697,19,4.59


In [20]:
data_df = data_df[data_df['rating'] != 0]

In [49]:
# Load the jokes
jokes_df = pd.read_excel('/home/featurize/Dataset4JokeSet.xlsx',header=None, names=['joke'])

In [92]:
# 首先，给这个未命名的列一个名字 'joke_text'
jokes_df.columns = ['joke']

# 然后创建一个新的 'joke_id' 列，您可以根据需要填充它
# 例如，我们这里简单地使用从1开始的整数作为joke_id
jokes_df['joke_id'] = range(1, len(jokes_df) + 1)

# 现在，我们可以将 'joke_id' 设置为索引
jokes_df.set_index('joke_id', inplace=True)

In [94]:
jokes_df

,joke
joke_id,
1,"A man visits the doctor. The doctor says ""I ha..."
2,This couple had an excellent relationship goin...
3,Q. What's 200 feet long and has 4 teeth? A. ...
4,Q. What's the difference between a man and a t...
5,Q.\tWhat's O. J. Simpson's Internet address? ...
...,...
154,"Poodle: ""My life is a mess. My owner is mean, ..."
155,Did you hear that NASA has launched several co...
156,"A bear walks into a bar and says,""I'd like a b..."


In [93]:
#完成实验步骤中对数据集操作后执行
jokes_df.head()

,joke
joke_id,
1,"A man visits the doctor. The doctor says ""I ha..."
2,This couple had an excellent relationship goin...
3,Q. What's 200 feet long and has 4 teeth? A. ...
4,Q. What's the difference between a man and a t...
5,Q.\tWhat's O. J. Simpson's Internet address? ...


In [57]:
# Create a Reader object
reader = Reader(rating_scale=(1, 10))

In [59]:
# 使用 reader 方法 load_from_df 来加载 DataFrame
data = Dataset.load_from_df(data_df[['user_id', 'joke_id', 'rating']], reader)

In [60]:
# 使用 train_test_split 函数划分数据集
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [61]:
# Train a SVD model
# 创建并训练 SVD 模型
algo = SVD()
algo.fit(trainset)

In [62]:
# Test the trained model against the testset
# 在测试集上进行预测
predictions = algo.test(testset)

In [64]:
# 计算并打印 RMSE
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

# 计算并打印 MAE
mae = accuracy.mae(predictions)
print(f"MAE: {mae}")

RMSE: 8.6310
RMSE: 8.63101488964829
MAE:  5.0595
MAE: 5.059464092406833


In [67]:
# Get the id of jokes
all_jokes_ids = data_df['joke_id'].unique()

In [68]:
# Choose an unused user_id for the new user
new_user_id = data_df['user_id'].max() + 1

In [69]:
# Add new user's ratings to the data
new_user_ratings = {
    "107": 10,
    "149": 9,
    "34": 2
}

In [70]:
# Create a Reader object
reader = Reader(rating_scale=(1, 10))

In [71]:
# 创建包含新用户评分的 DataFrame
new_user_data_df = pd.DataFrame({
    'user_id': [new_user_id] * len(new_user_ratings),
    'joke_id': list(new_user_ratings.keys()),
    'rating': list(new_user_ratings.values())
})

In [72]:
# 将新用户的评分添加到原始数据中
data_df = pd.concat([data_df, new_user_data_df], ignore_index=True)

In [73]:
# Load the data into a Dataset object
# 加载数据到 Dataset 对象
data = Dataset.load_from_df(data_df[['user_id', 'joke_id', 'rating']], reader)

In [74]:
# Build a full trainset from data
# 划分训练集和测试集
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [75]:
# Train a SVD model
# Train a SVD model
# 创建并训练 SVD 模型
algo = SVD()
algo.fit(trainset)

In [ ]:
# Generate recommendations for the new user


In [76]:
# Predict the ratings for all unrated jokes
# 预测新用户对所有未评分笑话的评分
unrated_jokes = [joke_id for joke_id in all_jokes_ids if joke_id not in new_user_ratings]
predictions = [algo.predict(new_user_id, joke_id) for joke_id in unrated_jokes]

In [77]:
# Get the top 10 jokes with highest predicted ratings
# 创建一个按预测评分降序排序的列表
predictions.sort(key=lambda x: x.est, reverse=True)

# 获取预测评分最高的前10个笑话
top_10_jokes = predictions[:10]

上方俩处代码存在问题

In [87]:
# 修复打印语句中的错误并执行打印
print("Top 10 joke recommendations for the new user:")
print([joke.iid for joke in top_10_jokes])

Top 10 joke recommendations for the new user:
[0, 129, 109, 134, 111, 89, 112, 32, 114, 137]


In [91]:
# 修复打印语句中的错误并执行打印
print("Top 10 joke recommendations for the new user:")
print(top_10_jokes)

Top 10 joke recommendations for the new user:
[Prediction(uid=7698, iid=0, r_ui=None, est=10, details={'was_impossible': False}), Prediction(uid=7698, iid=129, r_ui=None, est=4.409912682545975, details={'was_impossible': False}), Prediction(uid=7698, iid=109, r_ui=None, est=4.117712012028947, details={'was_impossible': False}), Prediction(uid=7698, iid=134, r_ui=None, est=4.019507389844497, details={'was_impossible': False}), Prediction(uid=7698, iid=111, r_ui=None, est=3.9998509466080194, details={'was_impossible': False}), Prediction(uid=7698, iid=89, r_ui=None, est=3.8418461882161985, details={'was_impossible': False}), Prediction(uid=7698, iid=112, r_ui=None, est=3.780439942263283, details={'was_impossible': False}), Prediction(uid=7698, iid=32, r_ui=None, est=3.6564134400491692, details={'was_impossible': False}), Prediction(uid=7698, iid=114, r_ui=None, est=3.6302614669786406, details={'was_impossible': False}), Prediction(uid=7698, iid=137, r_ui=None, est=3.5826631630979966, det

In [ ]:
结果呈现有问题，代码存在问题
+7 基础分